# Howto Polars
- Venis de Pandas y querés aprender a usar Polars
- Veamos las diferencias y similitudes en las tareas más comunes

- La diferencia mas importante:
   - Polars establece operaciones que no son ejecutadas hasta no tener un contexto (algo que en pandas no existe).
   - Por tanto, uno opera con columnas genéricas de ningún dataframe (`pl.col`) en particular
   - Luego, cuando se les da contexto, es decir, se las relaciona con un dataframe en concreto, ahí se ejecutan y toman sentido ([Ver ejemplo de la documentación oficial](https://docs.pola.rs/user-guide/concepts/expressions-and-contexts/#expressions)).

```
imc = pl.col("peso")/pl.col("altura")**2
```

In [1]:
import polars as pl
import pandas as pd

## Lectura de CSV  

In [3]:
pan_df = pd.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv")
pan_df.head()

,datetime,timelinux,s1
0,2022-11-17 17:04:39,1.668705,104.0
1,2022-11-17 17:34:37,1.668706,107.0
2,2022-11-17 18:04:38,1.668708,3.0
3,2022-11-17 18:34:38,1.668710,3.0
4,2022-11-17 19:04:37,1.668712,2.0


In [5]:
# En pandas con .info() conocemos el tipo de dato de cada columna 
pan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   1130 non-null   object 
 1   timelinux  1130 non-null   float64
 2   s1         1130 non-null   float64
dtypes: float64(2), object(1)
memory usage: 26.6+ KB


In [7]:
pol_df = pl.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv")
# En polars ya nos informa el tipo de dato de cada columna
pol_df.head()

datetime,timelinux,s1
str,f64,f64
"""2022-11-17 17:04:39""",1.668705,104.0
"""2022-11-17 17:34:37""",1.668706,107.0
"""2022-11-17 18:04:38""",1.668708,3.0
"""2022-11-17 18:34:38""",1.66871,3.0
"""2022-11-17 19:04:37""",1.668712,2.0


In [13]:
# Polars con .schema nos informacada columna el tipo de dato asociado
pol_df.schema

Schema([('datetime', String), ('timelinux', Float64), ('s1', Float64)])

### Leer columna como datetime

- En pandas podemos indicar que una columna es de ese tipo al leer el archivo, en polars?

In [16]:
pan_df = pd.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv", parse_dates = ["datetime"])
pan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   1130 non-null   datetime64[ns]
 1   timelinux  1130 non-null   float64       
 2   s1         1130 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 26.6 KB


- **Polars** tiene `try_parse_dates` que se da cuenta del formato y ya lo interpreta correctamente, fijate: 

In [18]:
pol_df = pl.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv", try_parse_dates=True)
pol_df.head()

datetime,timelinux,s1
datetime[μs],f64,f64
2022-11-17 17:04:39,1.668705,104.0
2022-11-17 17:34:37,1.668706,107.0
2022-11-17 18:04:38,1.668708,3.0
2022-11-17 18:34:38,1.66871,3.0
2022-11-17 19:04:37,1.668712,2.0


- En **Polars** también se la podemos indicar manualmente con `schema_overrides` (antes era `dtypes`), así (ojo, tiene que ser compatible, sino se rompe):

In [22]:
pol_df = pl.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv", 
                     schema_overrides={"datetime": pl.Datetime})
pol_df.head()

datetime,timelinux,s1
datetime[μs],f64,f64
2022-11-17 17:04:39,1.668705,104.0
2022-11-17 17:34:37,1.668706,107.0
2022-11-17 18:04:38,1.668708,3.0
2022-11-17 18:34:38,1.66871,3.0
2022-11-17 19:04:37,1.668712,2.0


## Seleccionar columnas 
- En pandas seleccionar una columna te retorna una Serie, en polars?

In [24]:
type(pan_df["datetime"])

pandas.core.series.Series

In [26]:
type(pol_df["datetime"])

polars.series.series.Series

- Seleccionamos varias columnas: igual que pandas

In [31]:
pol_df[["datetime", "s1"]]

datetime,s1
datetime[μs],f64
2022-11-17 17:04:39,104.0
2022-11-17 17:34:37,107.0
2022-11-17 18:04:38,3.0
2022-11-17 18:34:38,3.0
2022-11-17 19:04:37,2.0
…,…
2022-12-11 04:34:25,103.0
2022-12-11 05:04:25,91.0
2022-12-11 05:34:25,86.0


## No tan distintos
- Vamos derecho a lo que es diferente

In [9]:
# Creamos un DataFrame a partir de una lista de diccionarios
data = [
    {"nombre": "Ana", "edad": 30, "ciudad": "Buenos Aires"},
    {"nombre": "Pedro", "edad": 25, "ciudad": "Córdoba"},
    {"nombre": "María", "edad": 35, "ciudad": "Buenos Aires"},
    {"nombre": "Emiliano", "edad": 43, "ciudad": "Tostado", "dni":"28331969"},
]

df = pl.DataFrame(data)

# Mostramos el DataFrame
print(df)

shape: (4, 4)
┌──────────┬──────┬──────────────┬──────────┐
│ nombre   ┆ edad ┆ ciudad       ┆ dni      │
│ ---      ┆ ---  ┆ ---          ┆ ---      │
│ str      ┆ i64  ┆ str          ┆ str      │
╞══════════╪══════╪══════════════╪══════════╡
│ Ana      ┆ 30   ┆ Buenos Aires ┆ null     │
│ Pedro    ┆ 25   ┆ Córdoba      ┆ null     │
│ María    ┆ 35   ┆ Buenos Aires ┆ null     │
│ Emiliano ┆ 43   ┆ Tostado      ┆ 28331969 │
└──────────┴──────┴──────────────┴──────────┘


In [15]:
# filtramos parecido a lo que serìa el boolean indexing de pandas

df_filtrado = df.filter(
    (pl.col("edad") >= 30) & (pl.col("ciudad") == "Buenos Aires")
)

df_filtrado

nombre,edad,ciudad,dni
str,i64,str,str
"""Ana""",30,"""Buenos Aires""",null
"""María""",35,"""Buenos Aires""",null


In [19]:
# tenemos metodos sobre las columnas cuando tenemos valores nulos

df_filtrado = df.filter(
    (pl.col("edad") >= 30) & (~pl.col("dni").is_null())
)

df_filtrado

nombre,edad,ciudad,dni
str,i64,str,str
"""Emiliano""",43,"""Tostado""","""28331969"""
